# 用机器学习方法完成中文文本分类

* 输入形态：

CountVectorizer <br>
bow : bag of words <br>
ti-idf : text features <br>
word2vec : cbow/skip-gram (单个维度没有物理含义) <br>
* 传统机器学习模型：

Naive Bayes <br>
SVM <br>
RF <br>
GBDT
* 深度学习模型：

MLP <br>
CNN <br>
LSTM

## 朴素贝叶斯

我们试试用朴素贝叶斯完成一个中文文本分类器，一般在数据量足够，数据丰富度够的情况下，用朴素贝叶斯完成这个任务，准确度还是很不错的。

机器学习的算法要取得好效果，离不开数据，咱们先把数据加载进来看看。

### 准备数据

准备好数据，我们挑选 科技、汽车、娱乐、军事、运动 总共5类文本数据进行处理。

In [9]:
# 导入库
import jieba
import pandas as pd

#停用词
stopwords_data_path = "data/stopwords.txt"
stopwords_data_df = pd.read_csv(stopwords_data_path,encoding="utf-8",sep="\t",index_col=None,quoting=3,names=["stopword"])

# 准备数据
df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_technology = df_technology.dropna()

df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_military = df_military.dropna()

df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_sports = df_sports.dropna()

stopwords = stopwords_data_df.stopword.values.tolist()
technology = df_technology.content.apply(lambda line: line.strip()).values.tolist()
car = df_car.content.apply(lambda line: line.strip()).values.tolist()
entertainment = df_entertainment.content.apply(lambda line: line.strip()).values.tolist()
military = df_military.content.apply(lambda line: line.strip()).values.tolist()
sports = df_sports.content.apply(lambda line: line.strip()).values.tolist()

我们抽取几天数据来看下数据内容

In [10]:
technology = df_technology.content.apply(lambda line:line.strip()).values.tolist()
car = df_car.content.apply(lambda line:line.strip()).values.tolist()
entertainment = df_entertainment.content.apply(lambda line:line.strip()).values.tolist()
military = df_military.content.apply(lambda line:line.strip()).values.tolist()
sports = df_sports.content.apply(lambda line:line.strip()).values.tolist()

In [11]:
technology[20]

'2016年12月底，IDC发布调查报告显示，中国手机厂商于去年10月份在印度最主要的30个城市拿下了40%的市场份额。尽管三星仍然占据第一的位置，但二三名都被中国手机厂商拿下。其中，三星市场份额为26.1%，较上月增长15.8%，联想(包括摩托罗拉)以13.4%的份额位居第二，较上个月增长了50%，小米以10.7%的份额位列第三，较上月增长41.7%。'

In [12]:
car[10]

'从长远来看，当前我国新能源汽车市场还处于“政策驱动”的阶段，伴随着政府补贴政策进入退坡通道，行业或将进入发展的调整期，这将倒逼新能源汽车企业强化对核心技术的突破创新，推动车企通过提高自身综合实力来降本增效，大力提升新能源汽车的性价比，在退坡的周期内逐渐走上依托市场自行发展的道路。'

In [13]:
entertainment[4]

'网络综艺与电视综艺在播出模式、观众群体以及节目板块等方面也都存在差异。在传播上，电视台比网络平台更容易产生爆款。尽管这几年电视综艺也在网站上线，但由于观众观看的时间相对一致，而网络节目的时间较随意，所以同一档节目在同一时间，电视聚集观众的能力要比互联网强。《爸爸去哪儿4》转成网络综艺播出时，虽然也因一些话题而引发讨论，但是因传统受众观看习惯的影响，而网络观看又具有随意性，缺乏即时的、一致的观看体验，而导致没有掀起前几季的全民讨论的热潮。'

In [14]:
military[100]

'火箭军某导弹旅参谋长 陈伟：'

In [15]:
sports[900]

'“明年会继续争取吗？”媒体追问。此时的小德情绪显然有了波动，“到时候再说吧。我理解大家的工作，但也请你们理解我现在的处境。”'

### 分词和中文处理

5类数据最终处理的格式：<br>
word1 word2 word3 technology <br>
word1 word2 word3 word4 technology <br>
word1 word2 word4 car <br>

In [16]:
'''
5类数据最终处理的格式：
word1 word2 word3 technology 
word1 word2 word3 word4 technology 
word1 word2 word4 car 
'''

'''
    lines: 文章的一条记录
    sentences: 返回的数据列表
    category： 文章的类别
'''
def preprocess_text(lines, file_name, category):

    with open(file_name,"w") as f:
        for line in lines:
            segs = jieba.lcut(line)
            segs = [seg for seg in segs if len(seg) > 1 and seg not in stopwords]
            data = " ".join(segs)
            label = category
            f.write(label + " " + data + "\n")

## 处理 label data 格式的数据
preprocess_text(technology, "data_sample/technology_sample.txt", "technology")
preprocess_text(entertainment, "data_sample/entertainment_sample.txt", "entertainment")
preprocess_text(car, "data_sample/car_sample.txt", "car")
preprocess_text(military, "data_sample/military_sample.txt", "military")
preprocess_text(sports, "data_sample/sports_sample.txt", "sports")

In [17]:
## 合并文件
merge_data_path = "data_sample/data_label.txt"
category_lst = ['technology','entertainment','car','military','sports']
with open(merge_data_path,"w") as f_write:
    for category in category_lst:
        file_path = "data_sample/{0}_sample.txt".format(category)
        with open(file_path) as f:
            for line in f.readlines():
                f_write.write(line)

In [19]:
## 打乱文件的顺序
documents = []
with open(merge_data_path,"r") as f:
    for line in f.readlines():
         documents.append(line);

import random
random.shuffle(documents)
for data in documents[0:10]:
    print(data.strip())

technology 时间 精度 空间 精度 得以 提高 时间 精度 光学 成像 功能性 核磁共振 FMRI 近红外 成像 NIRS 遗憾 两种 成像 原理 测量 含量 变化 捕捉 单词 显得 力不从心 近红外 成像 为例 神经 活动 脑区 血流量 耗氧量 增加 脱氧 血红蛋白 含氧 血红蛋白 对光 吸收 程度 对光 吸收 大脑皮层 代谢 情况 耦合 脑区 活动 神经 活动 单位 毫秒 单位 动作电位 钠离子 细胞 离子 排出 细胞 过程 这套 成像 系统 优点 追踪 测量 动作电位 中钠 离子 浓度 变化 光学 特征 毫秒 时间 测量 精度
entertainment 样片 泄露 影视剧 票房 收视率 很大 影响 中闻 律师 事务所律师 赵虎 版权 电视剧 作品 享有 发表权 放映 广播 版权 发表 东西 未经许可 发表 一种 侵犯 著作权 侵权人 营利 目的 侵犯 著作权 触犯 刑法 侵犯 著作权
military 英烈 时代 楷模 家庭 子女 妻子 丈夫 英雄 舍生忘死 生命 缺少 分量 热爱生活 热爱 家庭 一群 透过 寻常 生活 一言一行 一颦一笑 英雄 建立 生命 连接 切近 感受 英雄 情怀 牺牲 可贵 令人 肃然起敬
technology 消费市场 加速 细分 品牌 精细化 应对
entertainment 繁华 京城 寒冬 未显 寂静 丽都 皇冠 假日酒店 突显出 喧嚣 喜悦 临近 春节 购置 年货 热闹 景象 上海滩 原貌 重现 遊戏 规则 首映礼 开启 导演 高希希 携手 何润东 黄子 王学圻 高捷 师鹏 陈冰 主创人员 齐聚 上海 现场 粉丝 媒体 近距离 互动 高涨 热情 现场 气氛 一次次 陷入 高潮
technology 广东电信 开发 天翼 蓝盾 系统 去年 广东省 试点 上线 系统 融合 数据 技术 网络 定位 网络 控制 触点 技术 各类 恶意 骚扰 高仿 诈骗 非法 号码 多个 异常 话务 分析 解决 诈骗 电话 短信 早期 主动 识别 主动防御 难题
sports 广东 绿色 山河 公司 操盘手 李勇鸿及 父亲 李乃志 大哥 李洪强 弟弟 李勇飞 2004 李乃志 李勇 飞因 非法 吸收 公众 存款 判处 有期徒刑 处罚金 李勇 立案 行踪 成谜
entertainment 活动 当天 北京 好莱坞 文化产业 传媒 董事长 琳